# GATES USING NAND

Using the simple quantum Toffoli gate, we designed a universal NAND gate such that when the target bit of the Toffoli gate is set to 1, it behaves like a NAND gate. Using this implementation we further defined basic AND and OR logic gates. The implementation of these basic gates opens a wide range of possibilities as it would enable us to design and implement virtually any logic circuit using the Braket.


##### Importing necessary Libraries

In [5]:
import boto3
from braket.circuits import Circuit
from braket.aws import AwsDevice
import matplotlib.pyplot as plt
from braket.devices import LocalSimulator

##### Accessing the AWS account and initialising the Braket device to run the circuits

In [6]:
aws_account_id = boto3.client("sts").get_caller_identity()["Account"]

In [7]:
#device = AwsDevice('arn:aws:braket:::device/quantum-simulator/amazon/sv1')
device = LocalSimulator()

In [8]:
#s3_folder = (f"amazon-braket-7aaad4cbb8e5","Simulator")

### NAND Gate

![alt text](images/nand.png "Title")

In [12]:
def NAND_GATE(a,b):
    final = []
    for i in range(len(a)):
        final.append(int(a[i]))
        final.append(int(b[i]))
        final.append(1)
    rev = []
    for i in range(len(final)):
        if final[i] == 1:
            rev.append(i)
    tofoli = Circuit().x(rev)
    for qubit in range(0,len(final)-2,3):
        tofoli.ccnot(qubit,qubit+1,qubit+2)
    counts = device.run(tofoli, shots = 1).result().measurement_counts
    res = ""
    for key, value in counts.items():
        for i in range(2,len(final),3):
            res += key[i]
    return res

- > **Tofoli Gate acts as a NAND gate when the target bit is set to 1.**
- > **Here we have implemented NAND Gate from Tofoli gate and simulated different Classical circuits using NAND as it is a Universal Gate.**

### AND Gate

![alt text](images/nand-gate6-2cb60496.png "Title")

In [16]:
def AND_GATE(a,b):
    out = NAND_GATE(a,b)
    return NAND_GATE(out,out)

In [18]:
a = input("Enter the first number: ")
b = input("Enter the second number: ")
out = AND_GATE(a,b)
print("{} AND {} = {}".format(a,b,out))

Enter the first number: 10110
Enter the second number: 11100
10110 AND 11100 = 10100


### OR Gate

![alt text](images/or-gate-using-nand-gates.png "Title")

In [20]:
def OR_GATE(a,b):
    c = NAND_GATE(a,a)
    d = NAND_GATE(b,b)
    return NAND_GATE(c,d)

In [21]:
a = input("Enter the first number: ")
b = input("Enter the second number: ")

out = OR_GATE(a,b)
print("{} OR {} = {}".format(a,b,out))

Enter the first number: 10110
Enter the second number: 11100
10110 OR 11100 = 11110


### HALF-ADDER

![alt text](images/halfadder.jpg "Title")

In [25]:
def HALF_ADDER(a,b):
    ab = NAND_GATE(a,b)
    c = NAND_GATE(a,ab)
    d = NAND_GATE(b,ab)
    sum = NAND_GATE(c,d)
    carry = NAND_GATE(ab,ab)
    return sum,carry

##### For 1-bit

In [28]:
a = input("Enter the first number: ")
b = input("Enter the second number: ")
sum,carry = HALF_ADDER(a,b)
print("Sum = {} and Carry = {} when {} and {} is added using Half Adder".format(sum,carry,a,b))

Enter the first number: 1
Enter the second number: 0
Sum = 1 and Carry = 0 when 1 and 0 is added using Half Adder


##### For n-bits

In [27]:
a = input("Enter the first number: ")
b = input("Enter the second number: ")
sum,carry = HALF_ADDER(a,b)
print("Sum = {} and Carry = {} when {} and {} is added using Half Adder".format(sum,carry,a,b))

Enter the first number: 111
Enter the second number: 111
Sum = 000 and Carry = 111 when 111 and 111 is added using Half Adder


- > **From the above calculation we can see that Half Adder cannot be used as a binary adder as it doesn't have provision for carry-in from the previous circuit when adding together multiple data bit**
- > **We can overcome this with the help of a Full Adder**

### FULL-ADDER

![alt text](images/fulladder.jpg "Title")

In [22]:
def FULL_ADDER(x,y,cin):
    xy = NAND_GATE(x,y)
    xxy = NAND_GATE(x,xy)
    yxy = NAND_GATE(y,xy)
    a = NAND_GATE(xxy,yxy)
    b = NAND_GATE(a,cin)
    ab = NAND_GATE(a,b)
    c = NAND_GATE(b,cin)
    sum = NAND_GATE(ab,c)
    carry = NAND_GATE(b,xy)
    return sum,carry
    

##### For 1-bit

In [29]:
a = input("Enter the first number: ")
b = input("Enter the second number: ")
cin = input("Enter the input carry: ")
sum,carry = FULL_ADDER(a,b,cin)
print("Sum = {} and Carry = {} when {} and {} is added using Full Adder with input carry {}".format(sum,carry,a,b,cin))

Enter the first number: 1
Enter the second number: 1
Enter the input carry: 1
Sum = 1 and Carry = 1 when 1 and 1 is added using Full Adder with input carry 1


##### For n-bits
### BINARY-ADDER

Full adder is a simple 1–bit adder. If we want to perform n–bit addition, then we need ‘n’ number of 1–bit full adders should be used in the form of a cascade connection.

In [51]:
def BINARY_ADDER(x,y,cin):
    sum = ""
    for i in range(len(x)):
        if i == 0:
            carry = cin
        out,carry = FULL_ADDER(x[-(i+1)],y[-(i+1)],carry)
        sum += out
    return sum[::-1],carry
    
   

In [53]:
a = int(input("Enter the first number: "))
b = int(input("Enter the second number: "))
x = bin(a).replace("0b", "")
y = bin(b).replace("0b", "")
x=x.zfill(max(len(x), len(y)))
y=y.zfill(max(len(x), len(y)))
sum,carry = BINARY_ADDER(x,y,'0')
out = carry+sum
out = int(out,2)
print("{} + {} = {}".format(a,b,out))

Enter the first number: 12
Enter the second number: 13
12 + 13 = 25
